# Set up project

## Create bucket

In the console we will do the following:
1. Navigate to Cloud Storage
1. Go to Buckets
1. Click Create
1. Follow the prompts, etc.

Instead, we will use the CLI to do this quickly with the following command:

In [ ]:
! gcloud storage buckets create "gs://mlops-maturity-tutorial-level0" --location='us-west1'

# Model training

## Develop model training script

For the purposes of this tutorial, we will define a very simple model that trains and saves a Random Forest model.

In [ ]:
import os
import pickle
from google.cloud import storage
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier


# Load the data
iris = load_iris()

# Train a basic Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(iris.data, iris.target)
print("Model fit complete")

# Save the model
with open("model.pkl", 'wb') as f:
    pickle.dump(model, f)
print("Model saved to GCS")

# Upload model artifact to Cloud Storage
model_directory = 'gs://mlops-maturity-tutorial-level0'
storage_path = os.path.join(model_directory, 'model.pkl')
blob = storage.blob.Blob.from_string(storage_path, client=storage.Client())
blob.upload_from_filename('model.pkl')
print("Model artifact pushed")

print("Training completed.")

## Submit training job

While yes, the above script runs here in a notebook, let's demonstrate how to use Vertex AI training to do this instead. This is especially the case for larger models that require a GPU or a fleet of GPUs where you don't want the Notebook instance to carry that load.

In the console, we would do the following:
1. Navigate to Vertex AI
1. Go to Training
1. Click Train New Model
1. Follow the prompts, etc.

Instead, we will use the Python SDK in order to automate the process. To do this, we do the following:
1. Save the above script as a .py (we have already done this in src/)
1. Use the Python SDK to create and submit a CustomTrainingJob

In [ ]:
from google.cloud.aiplatform import CustomTrainingJob

job = CustomTrainingJob(
    display_name='iris_level0',
    script_path='src/train.py',
    container_uri='us-docker.pkg.dev/vertex-ai/training/sklearn-cpu.1-0:latest',
    staging_bucket='mlops-maturity-tutorial-level0',
    location='us-west1'
)

job.submit()

# Model deployment

## Wrap model artifact

In the above script, we trained a model and outputted the model artifact as a model.pkl. Here we will load the model.pkl into an inference container.

In the console, we would do the following:
1. Navigate to Vertex AI
1. Go to Model Registry
1. Click Import
1. Follow the prompts

Instead, we will use the Python SDK in order to automate the process.

In [ ]:
from google.cloud.aiplatform import Model

model = Model.upload(
    display_name='iris_level0',
    serving_container_image_uri='us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-3:latest',
    artifact_uri='gs://mlops-maturity-tutorial-level0'
)

## Deploy model

Now we can deploy the trained model to an endpoint.

In the console, we would do the following:
1. Navigate to Vertex AI
1. Go to Model Registry
1. Find and click on the new model
1. Go to Deploy & Test
1. Click Deploy to Endpoint
1. Follow the prompts

Instead, we will use the Python SDK in order to automate the process.

In [ ]:
endpoint = model.deploy()

# Use endpoint

Finally, we can send a payload to the deployed endpoint.

In the console, we would do the following:
1. Navigate to Vertex AI
1. Go to Model Registry
1. Find and click on the new model
1. Go to Deploy & Test
1. Select the new endpoint
1. Test the model with the input box.

Instead, we will use the Python SDK in order to automate the process.

In [ ]:
instances = [
    [1, 2, 3, 4],
    [5, 6, 7, 5]
]

responses = endpoint.predict(instances)

print(responses.predictions)

# Clean up

In [ ]:
# Undeploy endpoint
endpoint.undeploy_all()